# Geração de instâncias

In [4]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
env = gp.Env()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [5]:
from udgp import M4, generate_random_instance

# 12345 (átomo redundante)

N = 8
instance = generate_random_instance(N)

instance.view_input()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
previous_a = []
inst = instance

while not inst.is_solved():
    print("tentativa")

    inst = instance

    m = M4(inst, n=4, max_gap=1e-2, previous_a=previous_a, env=env)
    # m.Params.TimeLimit = 120
    m.optimize()
    inst = m.instance

    if m.Status is not GRB.OPTIMAL:
        continue
    
    previous_a.append(list(m.a_ijk_values()))

    print(list(m.a_ijk_values()))

    while inst.coords.shape[0] < inst.n:
        # print("step")
        m = M4(inst, n=1, max_gap=1e-2, env=env)
        m.setParam('TimeLimit', 60)
        m.optimize()

        if m.Status is not GRB.OPTIMAL:
            break

        inst = m.instance

inst.view()


tentativa

Interrupt request received
tentativa

Interrupt request received
tentativa
